In [2]:
import labMTsimple.storyLab 
from labMTsimple.storyLab import emotionFileReader
from labMTsimple.speedy import LabMT
from tqdm import trange
import pandas as pd
import codecs
import re
import seaborn as sns
from numpy import floor,zeros,array
import numpy as np
import shutil
import subprocess
import datetime

ModuleNotFoundError: No module named 'labMTsimple'

In [3]:
import labMTsimple

ModuleNotFoundError: No module named 'labMTsimple'

In [123]:
#lexicon = pd.read_csv('./NRC-VAD-Lexicon-Aug2018Release/NRC-VAD-Lexicon-Aug2018Release/NRC-VAD-Lexicon.txt', sep='\t')
lexicon = pd.read_csv("NRC-VAD-Lexicon.txt", sep = "\t")

In [124]:
lexicon['Ranking'] = np.arange(1, len(lexicon)+1)

In [125]:
columnsTitles = ["Word","Ranking","Arousal","Valence","Dominance"]
lexicon = lexicon.reindex(columns = columnsTitles)

In [126]:
avd_dict = lexicon.set_index('Word').T.to_dict('list')

In [127]:
nlist = lexicon['Valence'].tolist() 
# print(nlist)

In [128]:
wlist = lexicon['Word'].tolist() 
# print(wlist)

In [129]:
labMT = avd_dict
labMTvector = nlist
labMTwordList = wlist 

In [163]:
def chopper(words,labMT,labMTvector,outfile, minSize): #,minSize=1000
  # print "now splitting the text into chunks of size 1000"
  # print "and printing those frequency vectors"
  minSize = ((len(words))//80)
  allFvec = []
  from numpy import floor
  for i in range(int(floor(len(words)/minSize))):
    chunk = str('')
    if i == int(floor(len(words)/minSize))-1:
      # take the rest
      # print 'last chunk'
      # print 'getting words ' + str(i*minSize) + ' through ' + str(len(words)-1)
      for j in range(i*minSize,len(words)-1):
        chunk += words[j]+str(' ')
    else:
      # print 'getting words ' + str(i*minSize) + ' through ' + str((i+1)*minSize)
      for j in range(i*minSize,(i+1)*minSize):
        chunk += words[j]+str(' ')
        # print chunk[0:10]
    textValence,textFvec = labMTsimple.storyLab.emotion(chunk,labMT,shift=True,happsList=labMTvector)
      # print chunk
    # print 'the valence of {0} part {1} is {2}'.format(rawbook,i,textValence)
        
    allFvec.append(textFvec)


  f = open(outfile,"w")
  if len(allFvec) > 0:
    print("writing out the file to {0}".format(outfile))
    f.write('{0:.0f}'.format(allFvec[0][0]))
    for k in range(1,len(allFvec)):
      f.write(',{0:.0f}'.format(allFvec[k][0]))
    for i in range(1,len(allFvec[0])):
      f.write("\n")
      f.write('{0:.0f}'.format(allFvec[0][i]))
      for k in range(1,len(allFvec)):
        f.write(',{0:.0f}'.format(allFvec[k][i]))
    f.close()
  else:
    print("\""*40)
    print("could not write to {0}".format(outfile))
    print("\""*40)
  # print "done!"

In [176]:
def precomputeTimeseries(fullVec,labMT,labMTvector,outfile):
  minWindows = 80 #10
  timeseries = [0 for i in range(len(fullVec[0]))]
  # print len(timeseries)

  textFvec = [0 for j in range(len(fullVec))]
  for i in range(0,minWindows//2):
    textFvec = [textFvec[j]+fullVec[j][i] for j in range(len(fullVec))]
    # print "adding point {0}".format(i)

  for i in range(minWindows//2,minWindows):
    # print "scoring"
    stoppedVec = labMTsimple.storyLab.stopper(textFvec,labMTvector,labMTwordList,stopVal=2.0)
    timeseries[i-minWindows//2] = labMTsimple.storyLab.emotionV(stoppedVec,labMTvector)
    # print "adding point {0}".format(i)
    textFvec = [textFvec[j]+fullVec[j][i] for j in range(len(fullVec))]

  for i in range(minWindows,len(timeseries)-1):
    # print "scoring"
    stoppedVec = labMTsimple.storyLab.stopper(textFvec,labMTvector,labMTwordList,stopVal=2.0)
    timeseries[i-minWindows//2] = labMTsimple.storyLab.emotionV(stoppedVec,labMTvector)
    # print "adding point {0}".format(i)
    # print "removing point {0}".format(i-minWindows)
    textFvec = [textFvec[j]+fullVec[j][i]-fullVec[j][i-minWindows] for j in range(len(fullVec))]

  for i in range(len(timeseries)-1,len(timeseries)+minWindows//2):
    # print "scoring"
    stoppedVec = labMTsimple.storyLab.stopper(textFvec,labMTvector,labMTwordList,stopVal=2.0)
    timeseries[i-minWindows//2] = labMTsimple.storyLab.emotionV(stoppedVec,labMTvector)
    # print "removing point {0}".format(i-minWindows)
    textFvec = [textFvec[j]-fullVec[j][i-minWindows] for j in range(len(fullVec))]
    
  # print "done"

  # print timeseries[0:11]
  # print timeseries[-11:]

  g = open(outfile,"w")
  g.write("{0}".format(timeseries[0]))
  for i in range(1,len(timeseries)):
    g.write(",")
    # g.write("{0:.5f}".format(timeseries[i]))
    g.write("{0}".format(timeseries[i]))
  g.write("\n")
  g.close()

In [177]:
def process():
  # windowSizes = [500,1000,2000,5000,10000]
  # movie = 'wiki_avengers_2012' # movie = 'new4_into_the_wild'
    
  words = [x.lower() for x in re.findall(r"[\w\@\#\'\&\]\*\-\/\[\=\;]+",raw_text_clean,flags=re.UNICODE)]
  lines = raw_text_clean.split("\n")
  kwords = []
  klines = []
  for i in range(len(lines)):
    if lines[i][0:3] != "<b>":
      tmpwords = [x.lower() for x in re.findall(r"[\w\@\#\'\&\]\*\-\/\[\=\;]+",lines[i],flags=re.UNICODE)]
      kwords.extend(tmpwords)
      klines.extend([i for j in range(len(tmpwords))])

  # avhapps = emotion(raw_text,labMT)
  print("length of the original parse")
  print(len(words))
  print("length of the new parse")
  print(len(kwords))
  # print(kwords)
  # print len(klines)
  # print(klines[0:20])
  
  windowSizes = [((len(words))//80)]

  for window in windowSizes:
    print(window)

    # print klines[0:(window/10)]
    breaks = [klines[window//10*i] for i in range(int(floor(float(len(klines))//window)))] #*10
    breaks[0] = 0
    # print [window/10*i for i in xrange(int(floor(float(len(klines))/window*10)))]
    # print breaks
    # print len(breaks)
    f = open("test_vectors/"+movie+"-breaks.csv","w") #/"+str(window)+"
    f.write(",".join(map(str,breaks)))
    f.close()
    chopper(kwords,labMT,labMTvector,"test_vectors/"+movie+".csv",minSize=window//80) #/"+str(window)+" #//10

    f = open("test_vectors/"+movie+".csv","r") #/"+str(window)+"
    fullVec = [list(map(int,line.split(","))) for line in f]
    f.close()

    # some movies are blank
    if len(list(fullVec)) > 0:
      if len(list(fullVec[0])) > 9:
        precomputeTimeseries(fullVec,labMT,labMTvector,"test_timeseries/"+movie+".csv") #/"+str(window)+"
    else:
      print("this movie is blank:")
      print(movie.title)
      movie.exclude = True
      movie.excludeReason = "movie blank"
    return kwords
# process()

In [178]:
import matplotlib.pyplot as plt

def process_movie():
    kwords = process()
    df_time_series = pd.read_csv("test_timeseries/"+movie+".csv") #/"+str(window)+"
    df_time_series = df_time_series.T
    df_time_series = df_time_series.reset_index()
    df_time_series = df_time_series.rename(columns={"index": "score"})
    # print(df_time_series)
    # df_time_series['score'] = df_time_series['score'].astype(float)
    # df_time_series = list(map(float, df_time_series))
    for i in range(len(df_time_series['score'])): 
        df_time_series['score'][i] = ''.join(df_time_series['score'][i].rsplit('.', 1))
#     print(df_time_series['score'])
    df_time_series['score'] = pd.to_numeric(df_time_series['score']) #, errors = "coerce") # Pechthida added errors = "coerce"
    plt.figure()
    plot = sns.lineplot(x=df_time_series.index/max(df_time_series.index), y="score", data=df_time_series)
    plt.close()
#     print(kwords)
    return plot, kwords
# process_movie()

In [179]:
# TEST: Read in text file and process into csv
# Marvel's The Avengers
f = codecs.open("./wiki_summaries/Marvel's_The_Avengers.txt", "r", "utf8")
raw_text_clean = f.read()
f.close()
movie = "Marvel's_The_Avengers" 
# window = [len(raw_text_clean)//80]
# print(window)
# print(raw_text_clean)
wiki_avengers = process_movie()

length of the original parse
652
length of the new parse
652
8
writing out the file to test_vectors/Marvel's_The_Avengers.csv


In [141]:
# TEST: Read in csv file and plot as lineplot
# Marvel's The Avengers
row100 = pd.read_csv("./test_timeseries/Marvel's_The_Avengers.csv") #wiki_timeseries
row100 = row100.T
row100 = row100.reset_index()
row100 = row100.rename(columns={"index": "score"})
row100['score'] = pd.to_numeric(row100['score']) #, errors = "coerce") # Pechthida added errors = "coerce"
sns.lineplot(x=row100.index/max(row100.index), y="score", data=row100)

ValueError: Unable to parse string "0.5913684210526317.1" at position 2

In [142]:
import glob
scripts = glob.glob('./wiki_summaries/*.txt') #wiki_summaries

In [143]:
scripts

['./wiki_summaries\\$9.99.txt',
 './wiki_summaries\\1,000_Times_Good_Night.txt',
 './wiki_summaries\\100_Bloody_Acres.txt',
 './wiki_summaries\\102_Dalmatians.txt',
 './wiki_summaries\\10_Cloverfield_Lane.txt',
 './wiki_summaries\\10_Minutes_Gone.txt',
 './wiki_summaries\\10_MPH.txt',
 './wiki_summaries\\10_Things_I_Hate_About_You.txt',
 './wiki_summaries\\10_to_Midnight.txt',
 './wiki_summaries\\127_Hours.txt',
 './wiki_summaries\\12_Strong.txt',
 './wiki_summaries\\13_Ghosts.txt',
 './wiki_summaries\\13_Going_on_30.txt',
 './wiki_summaries\\13_Rue_Madeleine.txt',
 './wiki_summaries\\13_Sins.txt',
 './wiki_summaries\\13_Tzameti.txt',
 './wiki_summaries\\14_Blades.txt',
 './wiki_summaries\\14_Hours.txt',
 './wiki_summaries\\15_Minutes.txt',
 './wiki_summaries\\16_Blocks.txt',
 './wiki_summaries\\16_Years_of_Alcohol.txt',
 './wiki_summaries\\17_Girls.txt',
 './wiki_summaries\\18_Again!.txt',
 './wiki_summaries\\20,000_Leagues_Under_The_Sea.txt',
 './wiki_summaries\\20th_Century_Women.tx

In [144]:
# The cell below takes 8 hours to finish processing ~5k text files and converting them to CSV files. 
# Best to let run overnight! 

In [146]:
results = []

for idx, script in enumerate(scripts):
    file = codecs.open(script, "r", "utf8")
    raw_text_clean = file.read()
    file.close()
    name = script.split("\\",1)[1].replace('-', '').replace('.txt', '')
    movie = name
    window = 80 # window = ((len(raw_text_clean))//80) # window = 1000
    try:
        idx = process_movie()
        results.append(idx)
    except IndexError:
        pass
    except ZeroDivisionError: 
        continue
    print(movie)

length of the original parse
91
length of the new parse
91
1
writing out the file to test_vectors/$9.99.csv
$9.99
length of the original parse
150
length of the new parse
150
1
writing out the file to test_vectors/1,000_Times_Good_Night.csv
1,000_Times_Good_Night
length of the original parse
589
length of the new parse
589
7
writing out the file to test_vectors/100_Bloody_Acres.csv
100_Bloody_Acres
length of the original parse
558
length of the new parse
558
6
writing out the file to test_vectors/102_Dalmatians.csv
102_Dalmatians
length of the original parse
719
length of the new parse
719
8
writing out the file to test_vectors/10_Cloverfield_Lane.csv
10_Cloverfield_Lane
length of the original parse
108
length of the new parse
108
1
writing out the file to test_vectors/10_Minutes_Gone.csv
10_Minutes_Gone
length of the original parse
150
length of the new parse
150
1
writing out the file to test_vectors/10_MPH.csv
10_MPH
length of the original parse
691
length of the new parse
691
8
wri

KeyboardInterrupt: 